## 🏠 House Prices -Hiperparametros


Os modelos lineares (Regressão Linear, Ridge e Lasso) são utilizados para capturar relações lineares entre as variáveis explicativas e o preço dos imóveis. Esses modelos assumem uma relação aditiva entre os atributos e a variável alvo, oferecendo simplicidade, interpretabilidade e baixo custo computacional.



## 1. Bibliotecas


In [1]:
# =====================================================
# 🏠 House Prices - Modelos Lineares
# =====================================================
import pandas as pd
from pathlib import Path
# Scikit-learn - Modelos lineares
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib
import matplotlib.pyplot as plt
import os

from sklearn.linear_model import LinearRegression
from scipy.stats import loguniform

from sklearn.model_selection import cross_validate
import numpy as np
import time


from setup_notebook import setup_path
setup_path()
from src.model_utils import *
import joblib
import matplotlib.pyplot as plt
import os


from sklearn.exceptions import ConvergenceWarning
import warnings


## 2. Dataload & preprocessamento

In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
temp = joblib.load(BASE /'src'/'preprocess_house_prices_v1.joblib')
PP=temp['preprocessador']
colnull_train = temp['colnull_train']

# =====================================================
# 📁 1. Leitura dos dados & Separação das bases
# =====================================================
DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv")
y_test  = pd.read_csv(DATA_DIR / "y_test_raw.csv")



In [3]:
#baseline
model_base = LinearRegression()
pipe_base  = pipe_models(model_base , PP)

pipe_base.fit(X_train, y_train)
baseline_scores0 = cross_val_score(pipe_base, X_train, y_train, cv=10, scoring='r2')

print(f"{'='*70}")
print(f"Baseline CV")
print(f"R²: {baseline_scores0.mean():.4f} ± {baseline_scores0.std():.4f}")

# 2. Testa a performance 
y_pred0 = pipe_base.predict(X_test)
res0 = metricas_model(y_test, y_pred0, 'Baseline (test)',write='on')

Baseline CV
R²: 0.7178 ± 0.2440
🤖 BASELINE (TEST)
MAE:    0.095
RMSE: 0.1826
R²:   0.8035


In [4]:
#Ridge exploratorio RD1
param_dist_ridge_1 = {
    'model__alpha': loguniform(1e-5, 1e3),
    'model__fit_intercept': [True, False],
    'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag']
}

search_ridge_1 = RandomizedSearchCV(
    pipe_ridge,
    param_distributions=param_dist_ridge_1,
    n_iter=50,
    cv=10,
    scoring='r2',
    random_state=42,
    n_jobs=1,
    verbose=0
)

start = time.time()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    search_ridge_1.fit(X_train, y_train)
end = time.time()

best_ridge_1 = search_ridge_1.best_estimator_

scores_ridge_1= cross_val_score(best_ridge_1, X_train, y_train, cv=10, scoring='r2')

print(f"{'='*70}")
print(f"Baseline CV")
print(f"R²: {scores_ridge_1.mean():.4f} ± {scores_ridge_1.std():.4f}")

# 2. Testa a performance 
y_pred0 = best_ridge_1.predict(X_test)
res0 = metricas_model(y_test, y_pred0, 'Ridge Exploratorio',write='on')

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/50:.2f} segundos")
print("📌 Melhores Parâmetros:")
print(search_ridge_1.best_params_)
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

Baseline CV
R²: 0.8379 ± 0.1144
🤖 RIDGE EXPLORATORIO
MAE:   0.0964
RMSE: 0.1346
R²:   0.8932
Tempo total: 132.79 segundos
Tempo por iteração: 2.66 segundos
📌 Melhores Parâmetros:
{'model__alpha': 17.279373898388343, 'model__fit_intercept': True, 'model__solver': 'svd'}

#Processo finalizado em: 14:28:43


In [8]:
#Ridge refinado RD2

param_dist_ridge_2 = {
    'model__alpha': loguniform(1e0, 1e2),
    'model__fit_intercept': [True, False],
    'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag']
}

search_ridge_2 = RandomizedSearchCV(
    pipe_ridge,
    param_distributions=param_dist_ridge_2,
    n_iter=50,
    cv=10,
    scoring='r2',
    random_state=42,
    n_jobs=1,
    verbose=0
)

start = time.time()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    search_ridge_2.fit(X_train, y_train)
end = time.time()

best_ridge_2 = search_ridge_2.best_estimator_

scores_ridge_2= cross_val_score(best_ridge_2, X_train, y_train, cv=10, scoring='r2')

print(f"{'='*70}")
print(f"Baseline CV")
print(f"R²: {scores_ridge_2.mean():.4f} ± {scores_ridge_1.std():.4f}")

# 2. Testa a performance 
y_pred0 = best_ridge_2.predict(X_test)
res0 = metricas_model(y_test, y_pred0, 'Ridge Exploratorio',write='on')

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/50:.2f} segundos")
print("📌 Melhores Parâmetros:")
print(search_ridge_2.best_params_)
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

Baseline CV
R²: 0.8380 ± 0.1144
🤖 RIDGE EXPLORATORIO
MAE:   0.0967
RMSE: 0.1347
R²:    0.893
Tempo total: 103.82 segundos
Tempo por iteração: 2.08 segundos
📌 Melhores Parâmetros:
{'model__alpha': 13.258269776216489, 'model__fit_intercept': True, 'model__solver': 'sag'}

#Processo finalizado em: 14:42:35


In [9]:
#Lasso exploratorio LS1
model_lasso = Lasso(random_state=42, max_iter=5000)
pipe_lasso  = pipe_models(model_lasso, PP)

param_dist_lasso_1 = {
    'model__alpha': loguniform(1e-6, 1e1),   
    'model__fit_intercept': [True, False],
    'model__selection': ['cyclic', 'random']
}

search_lasso_1 = RandomizedSearchCV(
    pipe_lasso,
    param_distributions=param_dist_lasso_1,
    n_iter=50,
    cv=10,
    scoring='r2',
    random_state=42,
    n_jobs=1,
    verbose=0
)

start = time.time()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    search_lasso_1.fit(X_train, y_train)
end = time.time()

best_lasso_1 = search_lasso_1.best_estimator_

scores_lasso_1 = cross_val_score(
    best_lasso_1, X_train, y_train, cv=10, scoring='r2'
)

print(f"{'='*70}")
print(f"Lasso Exploratório CV")
print(f"R²: {scores_lasso_1.mean():.4f} ± {scores_lasso_1.std():.4f}")

# =====================================================
# 🧪 Avaliação em teste
# =====================================================
y_pred1 = best_lasso_1.predict(X_test)
res1 = metricas_model(y_test, y_pred1, 'Lasso Exploratório', write='on')

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/50:.2f} segundos")

print("📌 Melhores Parâmetros:")
print(search_lasso_1.best_params_)

print("\n# Processo finalizado em:", time.strftime("%H:%M:%S"))

Lasso Exploratório CV
R²: 0.8326 ± 0.1217
🤖 LASSO EXPLORATÓRIO
MAE:   0.0982
RMSE:  0.137
R²:   0.8894
Tempo total: 109.53 segundos
Tempo por iteração: 2.19 segundos
📌 Melhores Parâmetros:
{'model__alpha': 0.0015577217702693043, 'model__fit_intercept': True, 'model__selection': 'random'}

# Processo finalizado em: 14:46:50


In [11]:
#Lasso exploratorio LS1
param_dist_lasso_2 = {
    'model__alpha': loguniform(1e-6, 1e-4),   
    'model__fit_intercept': [True],
    'model__selection': ['random']
}

search_lasso_2 = RandomizedSearchCV(
    pipe_lasso,
    param_distributions=param_dist_lasso_2,
    n_iter=50,
    cv=10,
    scoring='r2',
    random_state=42,
    n_jobs=1,
    verbose=0
)

start = time.time()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    search_lasso_2.fit(X_train, y_train)
end = time.time()

best_lasso_2 = search_lasso_2.best_estimator_

scores_lasso_2 = cross_val_score(
    best_lasso_2, X_train, y_train, cv=10, scoring='r2'
)

print(f"{'='*70}")
print(f"Lasso refine CV")
print(f"R²: {scores_lasso_2.mean():.4f} ± {scores_lasso_2.std():.4f}")

# =====================================================
# 🧪 Avaliação em teste
# =====================================================
y_pred2 = best_lasso_2.predict(X_test)
res2 = metricas_model(y_test, y_pred2, 'Lasso Exploratório', write='on')

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/50:.2f} segundos")

print("📌 Melhores Parâmetros:")
print(search_lasso_2.best_params_)

print("\n# Processo finalizado em:", time.strftime("%H:%M:%S"))

Lasso Exploratório CV
R²: 0.7961 ± 0.1437
🤖 LASSO EXPLORATÓRIO
MAE:   0.0925
RMSE: 0.1388
R²:   0.8864
Tempo total: 358.87 segundos
Tempo por iteração: 7.18 segundos
📌 Melhores Parâmetros:
{'model__alpha': 8.706020878304853e-05, 'model__fit_intercept': True, 'model__selection': 'random'}

# Processo finalizado em: 14:57:33
